In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
# Import helpful libraries
import pandas as pd
import xgboost
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor   # For regression task
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

# Load the data, and separate the target
iowa_file_path = '/kaggle/input/home-data-for-ml-course/train.csv'
home_data = pd.read_csv(iowa_file_path)
y = home_data.SalePrice

features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 
    'MSSubClass',
    'OverallQual',
    'OverallCond',
    'YearRemodAdd',
    'GrLivArea',
    'HalfBath',
    'KitchenAbvGr',
    'Fireplaces',
    'WoodDeckSF',
    'OpenPorchSF',
    'EnclosedPorch',
    '3SsnPorch',
    'ScreenPorch',
    'PoolArea']
            
            
# Select columns corresponding to features, and preview the data
X = home_data[features]
X.head(5)


,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd,MSSubClass,OverallQual,OverallCond,...,GrLivArea,HalfBath,KitchenAbvGr,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea
0,8450,2003,856,854,2,3,8,60,7,5,...,1710,1,1,0,0,61,0,0,0,0
1,9600,1976,1262,0,2,3,6,20,6,8,...,1262,0,1,1,298,0,0,0,0,0
2,11250,2001,920,866,2,3,6,60,7,5,...,1786,1,1,1,0,42,0,0,0,0
3,9550,1915,961,756,1,3,7,70,7,5,...,1717,0,1,1,0,35,272,0,0,0
4,14260,2000,1145,1053,2,4,9,60,8,5,...,2198,1,1,1,192,84,0,0,0,0


In [3]:
from sklearn.feature_selection import mutual_info_regression

# Create a boolean mask for discrete features (int columns)
discrete_features = X.dtypes == int

# Function to calculate mutual information scores
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

# Calculate MI scores and display a few features with their MI scores
mi_scores = make_mi_scores(X, y, discrete_features)
print(mi_scores)  # Show a few features with their MI scores


OverallQual      5.841031e-01
GrLivArea        4.325460e-01
YearBuilt        4.099677e-01
LotArea          3.945098e-01
MSSubClass       2.870274e-01
1stFlrSF         2.843996e-01
YearRemodAdd     2.795163e-01
FullBath         2.581312e-01
TotRmsAbvGrd     2.116646e-01
OpenPorchSF      1.917840e-01
2ndFlrSF         1.722717e-01
Fireplaces       1.647983e-01
OverallCond      1.105338e-01
WoodDeckSF       1.041976e-01
HalfBath         8.304112e-02
BedroomAbvGr     7.852907e-02
EnclosedPorch    2.405439e-02
KitchenAbvGr     2.220382e-02
ScreenPorch      1.849359e-02
3SsnPorch        2.297872e-03
PoolArea         2.220446e-16
Name: MI Scores, dtype: float64


In [4]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Define a random forest model
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y)

result = permutation_importance(rf_model, X, y, n_repeats=10, random_state=42, n_jobs=-1)
importance_scores = pd.DataFrame({
    'Feature': val_X.columns,
    'Importance': result.importances_mean,
    'Std': result.importances_std
})
importance_scores = importance_scores.sort_values(by='Importance', ascending=False)
print(importance_scores)

          Feature  Importance       Std
8     OverallQual    0.559187  0.022659
11      GrLivArea    0.171083  0.005201
2        1stFlrSF    0.106920  0.004271
1       YearBuilt    0.045512  0.001078
0         LotArea    0.040595  0.001753
3        2ndFlrSF    0.028223  0.000429
10   YearRemodAdd    0.022896  0.000845
9     OverallCond    0.012391  0.000951
6    TotRmsAbvGrd    0.010138  0.000967
16    OpenPorchSF    0.009852  0.000340
15     WoodDeckSF    0.007229  0.000676
14     Fireplaces    0.007136  0.000546
7      MSSubClass    0.003998  0.000303
4        FullBath    0.003225  0.000269
19    ScreenPorch    0.001848  0.000221
5    BedroomAbvGr    0.001672  0.000195
12       HalfBath    0.001326  0.000108
13   KitchenAbvGr    0.000618  0.000046
18      3SsnPorch    0.000573  0.000241
17  EnclosedPorch    0.000473  0.000047
20       PoolArea    0.000264  0.000030


In [5]:
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 17,356


Validation MAE for Random Forest Model: 17,747
Validation MAE for Random Forest Model: 18,140

In [6]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = XGBRegressor(random_state=1)

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=1, ...)

In [7]:
# path to file you will use for predictions
test_data_path = '/kaggle/input/home-data-for-ml-course/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = test_data[features]

# make predictions which we will submit. 
test_preds = rf_model_on_full_data.predict(test_X)

In [8]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)